In [1]:
import os 
os.chdir("../..")
from MSML.src.math_spec_mapping import (load_from_json, create_action_chains_graph, write_out_boundary_actions,
                      write_out_policies, write_out_mechanisms, load_svg_graphviz, write_basic_report_full,
                      write_action_chain_reports, write_spec_tree, create_parameter_impact_table,
                     write_entity_reports, write_wiring_report, write_entity_markdown_report,
                     write_all_markdown_reports)
os.chdir("Quai-Macro-Model/Spec")

from IPython.display import display, display_html
from copy import deepcopy
from src import math_spec_json
import numpy as np
from pprint import pprint
from simulation import state_base, params_base, vesting_schedule_translate, build_logistic_classifier

ms = load_from_json(deepcopy(math_spec_json))
msi = ms.build_implementation(params_base)

Placeholder has no control action options


## Sanity Check Scenario

Premise: Ensure the simulation framework is operating as intended, without complications from the controller.

Condition(s): Fix the values of:
- $\mathbf u$, the controller update parameters ($k_{qi}, k_{quai}$); and
- $\pmb{\beta}$, the _miner decision population parameter vector_, i.e. the parameter vector for the miner's probabilistic decision on which token to receive as block reward.

Method: MC runs (across realizations of other stochastic variables), generating time series for e.g. miner decisions and proposed block rewards. 

In [2]:
state, params = msi.prepare_state_and_params(state_base, params_base, state_preperation_functions=[vesting_schedule_translate,
                                                                                                   build_logistic_classifier], parameter_preperation_functions=[])


for _ in range(100):
    # Set fixed values
    state["K Qi"] = 1
    state["K Quai"] = 500
    state["Population Mining Beta Vector"] = np.array([0, .001])


    msi.wiring["Mine Block Wiring"](state, params, [])
    msi.mechanisms["Log Simulation Data Mechanism"](state, params, [])

In [7]:
import pandas as pd
df = pd.DataFrame(state["Simulation History Log"])

df

,Estimate Beta0,Estimate Beta1,Population Beta0,Population Beta1
0,14.910537,1214.669030,0.0,0.001
1,14.807502,-109.158057,0.0,0.001
2,24.509804,411.199674,0.0,0.001
3,24.342746,640.964037,0.0,0.001
4,14.506770,1459.926531,0.0,0.001
...,...,...,...,...
95,74.805506,1346.274055,0.0,0.001
96,62.574493,-361.996466,0.0,0.001
97,62.314540,-382.853475,0.0,0.001
98,62.056738,-1193.816195,0.0,0.001
